In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import sys
sys.path.append("..")
sys.path.append("../..")
import numpy as np
import torch
from provider import CameraPoseProvider
import trimesh
from trimesh.viewer import scene_to_notebook
from omegaconf import OmegaConf
from gs.sh_renderer import SHRenderer
from utils.misc import save_img

In [ ]:
cfg = OmegaConf.load("../../conf/generative.yaml")
c2ws = []
camera_infos = []
provider = CameraPoseProvider(cfg)
for i in range(1000):
    out = provider.sample_one(1000)
    c2ws.append(out[1])
    camera_infos.append(out[0])

pos = torch.stack(c2ws, dim=0)[:, :3, 3].numpy()

In [ ]:
def visualize_poses(poses, size=0.1, show=False):
    # poses: [B, 4, 4]

    axes = trimesh.creation.axis(axis_length=4)
    box = trimesh.primitives.Box(extents=(2, 2, 2)).as_outline()
    box.colors = np.array([[128, 128, 128]] * len(box.entities))
    objects = [axes, box]

    for pose in poses:
        # a camera is visualized with 8 line segments.
        pos = pose[:3, 3]
        a = pos + size * pose[:3, 0] + size * pose[:3, 1] + size * pose[:3, 2]
        b = pos - size * pose[:3, 0] + size * pose[:3, 1] + size * pose[:3, 2]
        c = pos - size * pose[:3, 0] - size * pose[:3, 1] + size * pose[:3, 2]
        d = pos + size * pose[:3, 0] - size * pose[:3, 1] + size * pose[:3, 2]

        dir = (a + b + c + d) / 4 - pos
        dir = dir / (np.linalg.norm(dir) + 1e-8)
        o = pos + dir * 3

        segs = np.array([[pos, a], [pos, b], [pos, c], [pos, d], [a, b], [b, c], [c, d], [d, a], [pos, o]])
        segs = trimesh.load_path(segs)
        segs.colors = np.array([[255, 0, 0]] * len(segs.entities))
        objects.append(segs)

    return trimesh.Scene(objects)

In [ ]:
poses = torch.stack(c2ws, dim=0).cpu().numpy()
poses_scene = visualize_poses(poses)
scene_to_notebook(poses_scene)

In [ ]:
print(poses[..., :1, :].shape)
pose4x4 = np.concatenate([poses, np.zeros_like(poses[..., :1, :])], axis=1)
axes = []
for pose in pose4x4:
    axis = trimesh.creation.axis(transform=pose, axis_length=0.2)
    axes.append(axis)

scene = trimesh.Scene(axes)
scene_to_notebook(scene)

In [ ]:
cfg = OmegaConf.load("../../conf/generative.yaml")
c2ws = []
camera_infos = []
provider = CameraPoseProvider(cfg)
for i in range(1000):
    out = provider.sample_one(1000)
    c2ws.append(out[1])
    camera_infos.append(out[0])

pos = torch.stack(c2ws, dim=0)[:, :3, 3].numpy()
scene = trimesh.Scene()
pcd = trimesh.points.PointCloud(vertices=pos, colors=np.zeros_like(pos, dtype=np.uint8))
scene.add_geometry(pcd)
n_obj_pts = 1000
object_pts = np.random.randn(n_obj_pts, 3) * cfg.camera_pose_provider.center_jittor_std + np.array(cfg.camera_pose_provider.center)
obj_pcd = trimesh.points.PointCloud(vertices=object_pts, colors=np.zeros_like(object_pts, dtype=np.uint8))
scene.add_geometry(obj_pcd)

pts = torch.load("../../tmp/debug/pts.pt").cpu().numpy()
rgb = (np.random.rand(*pts.shape) * 255.).astype(np.uint8)

ppcd = trimesh.points.PointCloud(vertices=pts, colors=rgb)
# scene.add_geometry(ppcd)
scene.add_geometry(trimesh.creation.axis())

xxx = np.linspace(0, 10, 1000)
yyy = np.zeros_like(xxx)
zzz = np.zeros_like(xxx)

x_axis = trimesh.points.PointCloud(np.stack([xxx, yyy, zzz], axis=1), colors=np.array([255, 0, 0]))
scene.add_geometry(x_axis)
x_axis = trimesh.points.PointCloud(np.stack([xxx, yyy, zzz], axis=1), colors=np.array([255, 0, 0]))

yyy = np.linspace(0, 10, 1000)
xxx = np.zeros_like(yyy)
zzz = np.zeros_like(yyy)
y_axis = trimesh.points.PointCloud(np.stack([xxx, yyy, zzz], axis=1), colors=np.array([0, 255, 0]))
scene.add_geometry(y_axis)

box = trimesh.primitives.Box(extents=(2, 2, 2)).as_outline()
box.colors = np.array([[128, 128, 128]] * len(box.entities))
scene.add_geometry(box)


# size = 10
# cams = []
# for pose in c2ws:
#     pose = pose.cpu().numpy()
#     pos = pose[:3, 3]
#     a = pos + size * pose[:3, 0] + size * pose[:3, 1] - size * pose[:3, 2]
#     b = pos - size * pose[:3, 0] + size * pose[:3, 1] - size * pose[:3, 2]
#     c = pos - size * pose[:3, 0] - size * pose[:3, 1] - size * pose[:3, 2]
#     d = pos + size * pose[:3, 0] - size * pose[:3, 1] - size * pose[:3, 2]

#     dir = (a + b + c + d) / 4 - pos
#     dir = dir / (np.linalg.norm(dir) + 1e-8)
#     o = pos + dir * 3

#     segs = np.array([[pos, a], [pos, b], [pos, c], [pos, d], [a, b], [b, c], [c, d], [d, a], [pos, o]])
#     segs = trimesh.load_path(segs)
#     cams.append(segs)

# scene.add_geometry(cams)

scene.show(viewer=None, flags={"axis": True, 'wireframe': True})
# trimesh.viewer.notebook.scene_to_notebook(scene, axis=True)

In [ ]:
cfg.alpha_init = 0.999
cfg.skip_frustum_culling = True
renderer = SHRenderer(cfg, torch.from_numpy(pts), torch.from_numpy(rgb)).to("cuda")
for i, (c2w, camera_info) in enumerate(zip(c2ws, camera_infos)):
    out = renderer(c2w.cuda(), camera_info)
    save_img(out, "../../tmp/debug", f"{i:03d}.png")